In [ ]:
pip install googlemaps

In [ ]:
pip install prettyprint

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [60]:
import os

In [47]:
os.mkdir('/content/drive/MyDrive/restaurant_data_bangladesh_csv')

In [49]:
os.chdir('/content/drive/MyDrive/restaurant_data_bangladesh_csv')

In [51]:
import csv

In [78]:
def initialize_csv(file_name):
  f = open(file_name, 'w')
  writer = csv.DictWriter(
      f, fieldnames=['name', 'latitude', 'longitude', 'rating', 'number of reviews', 'affluence'])
  writer.writeheader()
  f.close()

In [53]:
def append_to_csv(file_name, fields):
  with open(file_name, 'a') as f:
    writer = csv.writer(f)
    writer.writerow(fields)

In [92]:
# initializing csv
file_name = 'restaurants.csv'
initialize_csv(file_name)

In [90]:
def send_data_to_csv(restaurants, file_name):
  for result in restaurants['results']:
    # extracting relevant data
    name = result['name']
    latitude = result['geometry']['location']['lat']
    longitude = result['geometry']['location']['lng']
    
    # check if each of the relevant key exists in the data
    rating = None 
    if 'rating' in result.keys():
      rating = result['rating']
    else:
      rating = '-'

    review_count = None
    if 'user_ratings_total' in result.keys():
      review_count = result['user_ratings_total']
    else: 
      review_count = 0
    
    price_level = None
    if 'price_level' in result.keys():
      price_level = '$' * result['price_level']
    else:
      price_level = '-'

    # writing data to csv file
    fields = [name, latitude, longitude, rating, review_count, price_level]
    append_to_csv(file_name, fields) 

In [93]:
import googlemaps
import time

# defining API Key
API_KEY = 'APIKEY'
google_map_api = googlemaps.Client(key=API_KEY)

# Using the latitude and longitude of Dhaka/Bangladesh and
# a radius of 50000m which is the maximum google places api will allow
restaurants = google_map_api.places_nearby(location='23.8118, 90.4134', radius=50000, type='restaurant')

send_data_to_csv(restaurants, file_name)

# wait for 3 seconds
time.sleep(3)

while 'next_page_token' in restaurants.keys():
  # get the next page of results
  restaurants = google_map_api.places_nearby(page_token=restaurants['next_page_token'])

  send_data_to_csv(restaurants, file_name)

  #wait 3 seconds
  time.sleep(3)